# Finite Capacity Region with Blocking (Multiclass Open Network)

This example creates a multiclass open network with 2 queues and a finite capacity region with global and per-class constraints. When the region is full, arriving jobs are blocked (wait in queue).

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
// Create network
val model = Network("FCR Blocking Example")

In [ ]:
// Add nodes
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")

In [ ]:
// Add job classes
val class1 = OpenClass(model, "Class1", 0)
val class2 = OpenClass(model, "Class2", 1)

In [ ]:
// Set arrival and service rates
source.setArrival(class1, Exp(0.4))
source.setArrival(class2, Exp(0.3))
queue1.setService(class1, Exp(1.0))
queue1.setService(class2, Exp(0.9))
queue2.setService(class1, Exp(1.1))
queue2.setService(class2, Exp(1.0))

In [ ]:
// Create routing matrix
val P = model.initRoutingMatrix()

// Class1 routing
P.set(class1, class1, source, queue1, 0.5)
P.set(class1, class1, source, queue2, 0.5)
P.set(class1, class1, queue1, queue2, 0.3)
P.set(class1, class1, queue1, sink, 0.7)
P.set(class1, class1, queue2, sink, 1.0)

// Class2 routing
P.set(class2, class2, source, queue1, 0.6)
P.set(class2, class2, source, queue2, 0.4)
P.set(class2, class2, queue1, queue2, 0.5)
P.set(class2, class2, queue1, sink, 0.5)
P.set(class2, class2, queue2, sink, 1.0)

model.link(P)

In [ ]:
// Add finite capacity region with constraints
// When region is full, jobs wait (blocking behavior)
val fcr = model.addRegion(listOf(queue1, queue2))
fcr.setGlobalMaxJobs(8)            // Global: max 8 jobs in region
fcr.setClassMaxJobs(class1, 5)     // Class1: max 5 jobs
fcr.setClassMaxJobs(class2, 4)     // Class2: max 4 jobs
fcr.setDropRule(class1, false)     // false = block (wait)
fcr.setDropRule(class2, false)     // false = block (wait)

In [ ]:
// Run JMT
val solver = JMT(model, "seed", 23000, "samples", 50000)
val avgTable = solver.avgTable
println("JMT Results:")
avgTable.print()

## Explanation

This example demonstrates a complex multiclass open queueing network with a Finite Capacity Region and blocking behavior:

1. **Network Structure**: Two queues (Queue1 and Queue2) with probabilistic routing between them.

2. **Finite Capacity Region**: Both queues are inside an FCR with:
   - Global limit: 8 jobs total
   - Class1 limit: 5 jobs
   - Class2 limit: 4 jobs

3. **Blocking Policy**: `setDropRule(false)` means jobs wait (block) when capacity limits are reached, rather than being lost.

4. **Multiclass Behavior**: Different classes have different arrival rates, service times, and routing probabilities.

The model is useful for studying systems where jobs can wait when the system is congested (e.g., manufacturing systems, computing clusters with admission control).